In [1]:
# %%
# The following script reads the event PairCreacted from the Uniswap V2 Factory contract and prints the list of all pairs created on the Uniswap V2 protocol.
# Imports
from web3 import Web3

In [2]:
# Connect to a local node

nodes = ['https://mainnet.infura.io/v3/174e045511c742af9cbe23f6bd053402',
         'https://mainnet.infura.io/v3/0731bcfffdb844daa0972c90e98da4e6',
         'https://mainnet.infura.io/v3/6d0c853857a249878418776d98bf732a']
NODE_URI = nodes[0]
w3 = Web3(Web3.HTTPProvider(NODE_URI))
w3local = Web3(Web3.HTTPProvider('http://localhost:8545'))


In [3]:
# Define the contract address
# Uniswap V2 factory contract address
# contract_address = '0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f'
# SushiSwap factory contract address
contract_address = '0xC0AEe478e3658e2610c5F7A4A2E1777cE9e4f2Ac'

In [4]:

# Define the contract ABI
factory_abi = [
    {
        "anonymous": False,
        "inputs": [
            {
                "indexed": True,
                "internalType": "address",
                "name": "token0",
                "type": "address"
            },
            {
                "indexed": True,
                "internalType": "address",
                "name": "token1",
                "type": "address"
            },
            {
                "indexed": False,
                "internalType": "address",
                "name": "pair",
                "type": "address"
            },
            {
                "indexed": False,
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "name": "PairCreated",
        "type": "event"
    }
]

In [5]:
# Instantiate the contract
factory_contract = w3.eth.contract(address=contract_address, abi=factory_abi)
# Get events from the contract
events = factory_contract.events.PairCreated().create_filter(fromBlock='0x0', toBlock='latest').get_all_entries()
print(f'Found {len(events)} events')

Found 3763 events


In [6]:
# %%
# Implement a function that overcomes the 10k elements limitation of infura.
def getPairEvents(contract, fromBlock, toBlock):
    # This function tries to fetch all the events between fromBlock and toBlock
    # If more than 10k events are found, the function recursively fetches the events in smaller time intervals until all the events are fetched.
    # The function returns a list of all the events fetched.
    # When the 10k limit is reached, get_all_entries() throws an error.
    
    if toBlock == 'latest':
        toBlockPrime = w3.eth.block_number
    else:
        toBlockPrime = toBlock

    fetchCount = 0
    
    # Then, recursively fetch events in smaller time intervals
    def getEventsRecursive(contract, _from, _to):
        try:
            events = contract.events.PairCreated().create_filter(fromBlock=_from, toBlock=_to).get_all_entries()
            print("Found ", len(events), " events between blocks ", _from, " and ", _to)
            nonlocal fetchCount 
            fetchCount += len(events)
            return events
        except ValueError:
            print("Too many events found between blocks ", _from, " and ", _to)
            midBlock = (_from + _to) // 2
            return getEventsRecursive(contract, _from, midBlock) + getEventsRecursive(contract, midBlock + 1, _to)
    
    return getEventsRecursive(contract, fromBlock, toBlockPrime)

In [7]:
# %%
# Run the recursive getPairevents() function
events = getPairEvents(factory_contract, 0, w3.eth.block_number)
print(f'Found {len(events)} events')

Found  3763  events between blocks  0  and  17756257
Found 3763 events


In [8]:
# %%
# Convert the events to a list of dictionaries
pairDataList = [{'token0': e['args']['token0'],
    'token1': e['args']['token1'],
    'pair': e['args']['pair']} for e in events]
print(f"Here is event #0: {pairDataList[0]}")

Here is event #0: {'token0': '0x6B3595068778DD592e39A122f4f5a5cF09C90fE2', 'token1': '0xdAC17F958D2ee523a2206206994597C13D831ec7', 'pair': '0x680A025Da7b1be2c204D7745e809919bCE074026'}


In [9]:
# by the time being I can skip the solidity that reads reserves from pairs and use an existing contract
queryContractAddress = "0x6c618c74235c70DF9F6AD47c6b5E9c8D3876432B"

In [10]:
# %%
# The following parts compile and upload the contract code to a local node to experiment without paying gas fees. Later however, we will use the contract already deployed on the mainnet.
# Be careful as the code will use the abi of the compiled contract, which will be different from the abi of the deployed contract if you make changes to it.
contractContent = """
//SPDX-License-Identifier: UNLICENSED
pragma solidity 0.8;

pragma experimental ABIEncoderV2;

interface IUniswapV2Pair {
    function token0() external view returns (address);
    function token1() external view returns (address);
    function getReserves() external view returns (uint112 reserve0, uint112 reserve1, uint32 blockTimestampLast);
}

// Batch query contract
contract UniswapFlashQuery {
    function getReservesByPairs(IUniswapV2Pair[] calldata _pairs) external view returns (uint256[3][] memory) {
        uint256[3][] memory result = new uint256[3][](_pairs.length);
        for (uint i = 0; i < _pairs.length; i++) {
            (result[i][0], result[i][1], result[i][2]) = _pairs[i].getReserves();
        }
        return result;
    }
    function getReservesByPairsYul(IUniswapV2Pair[] calldata _pairs) external view returns (bytes32[] memory) {
        bytes32[] memory result = new bytes32[](_pairs.length * 3);

        assembly {
            let size := 0x60 // Size of the return data (reserve0, reserve1, blockTimestampLast)
            let callData := mload(0x40) // Allocate memory for the function selector
            mstore(callData, 0x0902f1ac00000000000000000000000000000000000000000000000000000000) // 4-byte function selector of the getReserves() function
            
            // Update the free memory pointer
            mstore(0x40, add(callData, 0x04))
            
            // let pairsCount := shr(0xe0, calldataload(sub(_pairs.offset, 0x20))) // Get the length of the _pairs array
            let pairsCount := _pairs.length

            for { let i := 0 } lt(i, pairsCount) { i := add(i, 1) } {
                // Load the pair address from the calldata
                let pair := calldataload(add(_pairs.offset, mul(i, 0x20)))
                    
                // Call the getReserves() function with preallocated memory for function selector
                let success := staticcall(gas(), pair, callData, 0x04, add(add(result, 0x20),mul(i, size)), size)
                if iszero(success) {
                    revert(0x00, 0x00)
                }
            }

            // Update the free memory pointer
            mstore(0x40, add(mload(0x40), mul(pairsCount, size)))
        }

        return result;
    }
}
"""

In [11]:
import solcx
version = "0.8.0"
filename = "UniswapFlashQuery.sol"
Output = "ir"
solcx.install_solc('0.8.0')   #ILDE

compiled_sol = solcx.compile_standard(
    {
        "language": "Solidity",
        "sources": {filename: {"content": contractContent}},
        "settings": {
            "outputSelection": {
                "*": {"*": ["abi", "metadata", "evm.bytecode", "evm.sourceMap",Output]}
            }
        },
    }#,
    #solc_version=version,
)

name = filename.split('.')[0]
res_bytecode = compiled_sol["contracts"][filename][name]["evm"]["bytecode"]["object"]
queryAbi = compiled_sol["contracts"][filename][name]["abi"]
# print(compiled_sol["contracts"][filename][name][Output])

# Export the bytecode
print(f"Bytecode: {res_bytecode[:100]}...")

solc, the solidity compiler commandline interface
Version: 0.8.0+commit.c7dfd78e.Linux.g++
Bytecode: 608060405234801561001057600080fd5b50610a64806100206000396000f3fe608060405234801561001057600080fd5b50...


In [47]:
# This code splits the request into chunks that are sent concurrently to the node.
import asyncio
from web3 import AsyncHTTPProvider
from web3.eth import AsyncEth
import time
# If you test this code in a Jupyter notebook, sligh modifications are needed like nest_asyncio.apply() (google for more info)
# [...]

# Create a function that takes a list of pair data, and returns a list of reserves for each pair
async def getReservesAsync(pairs, chunkSize=1000):
    # Create an async web3 provider instance
    w3Async = Web3(AsyncHTTPProvider(NODE_URI), modules={'eth': (AsyncEth)})

    # Create contract object
    queryContract = w3Async.eth.contract(address=queryContractAddress, abi=queryAbi)

    # Create a list of chunks of pair addresses
    chunks = [[pair['pair'] for pair in pairs[i:i + chunkSize]] for i in range(0, len(pairs), chunkSize)]

    # Gather all the async tasks
    tasks = [queryContract.functions.getReservesByPairs(pairs).call() for pairs in chunks]

    # Run the tasks in parallel
    results = await asyncio.gather(*tasks)

    return results

In [48]:
# Call the function

# next 2 lines from: https://stackoverflow.com/questions/46827007/runtimeerror-this-event-loop-is-already-running-in-python
import nest_asyncio
nest_asyncio.apply()

t0 = time.time()
reserves = asyncio.get_event_loop().run_until_complete(getReservesAsync(pairDataList))
print(f"Time taken: {time.time() - t0} seconds. Fetched {len(reserves)} reserves.")

Time taken: 3.5658562183380127 seconds. Fetched 4 reserves.


In [50]:
# This code sends the request chunks to multiple nodes in parallel.

# Define a list of node URIs
NODE_URIS = ['https://mainnet.infura.io/v3/174e045511c742af9cbe23f6bd053402',
             'https://mainnet.infura.io/v3/0731bcfffdb844daa0972c90e98da4e6',
             'https://mainnet.infura.io/v3/6d0c853857a249878418776d98bf732a']

providerList = [Web3(AsyncHTTPProvider(uri), modules={'eth': (AsyncEth)}) for uri in NODE_URIS]

async def getReservesParallel(pairs, providers, chunkSize=1000):
    # Create the contract objects
    contracts = [provider.eth.contract(address=queryContractAddress, abi=queryAbi) for provider in providers]

    # Create a list of chunks of pair addresses
    chunks = [[pair["pair"] for pair in pairs[i:i + chunkSize]] for i in range(0, len(pairs), chunkSize)]

    # Assign each chunk to a provider in a round-robin fashion
    tasks = [contracts[i % len(contracts)].functions.getReservesByPairs(pairs).call() for i, pairs in enumerate(chunks)]

    # Run the tasks in parallel
    results = await asyncio.gather(*tasks)
    
    # Flatten the results
    results = [item for sublist in results for item in sublist]

    return results

# Call the function
t0 = time.time()
reserves = asyncio.get_event_loop().run_until_complete(getReservesParallel(pairDataList, providerList))
print(f"Time taken: {time.time() - t0} seconds. Fetched {len(reserves)} results")

Time taken: 3.139055013656616 seconds. Fetched 3763 results
